In [1]:
import numpy as np
import torch
import scanpy as sc
import sklearn

import scvi
from scvi.data import AnnDataManager
from scvi.data.fields import LayerField, CategoricalObsField
from scvi.dataloaders import AnnDataLoader

from scvi.distributions import NegativeBinomial

Global seed set to 0


In [2]:
eb_adata = sc.read_h5ad("/project2/gilad/jpopp/ebQTL/data/single_cell_objects/Lowpass.3seqbatches.merged.1ksubsampled.raw.h5ad")

In [3]:
eb_adata

AnnData object with n_obs × n_vars = 16669 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'DROPLET.TYPE', 'SNG.POSTERIOR', 'NUM.SNPS', 'individual', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'FREE.TYPE', 'FREE.SNG.POSTERIOR', 'FREE.NUM.SNPS', 'free.individual', 'ALL.TYPE', 'ALL.SNG.POSTERIOR', 'ALL.NUM.SNPS', 'all.individual', 'c.date', 'Feeder.passage', 'Feeder.Free.passage', 'sex', 'Replicate', 'formation.date', 'visual.confluence', 'total.cells.per.lane..correct.', 'X..Viable..based.on.average.of.two.counts.', 'days.maintained.in.LD.prior.to.formation', 'who.dissociated.the.lines.for.formation', 'cell.counter.correction', 'comments', 'collection.date', 'EB.appearance.notes', 'dissociated.by.', 'approx..time.in.accumax..in.minutes.', 'Collection.Date', 'Lane', 'Sample.ID', 'cDNA.amp.cycles', 'cDNA.total.yield', 'library.prep.batch', 'Sequencing.batch', 'col.group', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',

In [4]:
anndata_fields = [
    LayerField(registry_key="x", layer=None, is_count_data=True),
    CategoricalObsField(registry_key="individual", obs_key="individual"),
]
adata_manager = AnnDataManager(fields=anndata_fields)
adata_manager.register_fields(eb_adata)
print(adata_manager.registry.keys())

dict_keys(['scvi_version', 'model_name', 'setup_args', 'field_registries', '_scvi_uuid'])


In [5]:
adata_manager.view_registry()

Anndata setup with scvi-tools version 0.15.0.

     Summary Statistics     
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Summary Stat Key ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│     n_cells      │ 16669 │
│      n_vars      │ 36601 │
│   n_individual   │  21   │
└──────────────────┴───────┘

                 Data Registry                  
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃      scvi-tools Location      ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      x       │            adata.X            │
│  individual  │ adata.obs['_scvi_individual'] │
└──────────────┴───────────────────────────────┘

                  individual State Registry                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location     ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['individual'] │   18501    │          0          │
│                         │   18502    │          1          │
│                         │   18505    │          2          │
│                         │   18517    │          3          │
│                         │   18520    │          4          │
│                         │   18522    │          5          │
│                         │   18856    │          6          │
│                         │   18870    │          7          │
│                         │   18907    │          8          │
│                         │   18912    │          9          │
│                         │   18913    │         10          │
│                         │   19093    │         11          │
│                         │   19101    │         12          │
│                         │   19108    │         13          │
│                         │   19116    │         14          │
│                         │   19140    │         15          │
│                         │   19153    │         16          │
│                         │   19159    │         17          │
│                         │   19190    │         18          │
│                         │   19203    │         19          │
│                         │   19210    │         20          │
└─────────────────────────┴────────────┴─────────────────────┘

In [6]:
# initialize an AnnDataLoader which will iterate over our anndata
adl = AnnDataLoader(adata_manager, shuffle=False, batch_size = 128)

# get the first batch of data
data_batch = next(iter(adl))

In [10]:
n_inds = adata_manager.summary_stats['n_individual']
n_genes = adata_manager.summary_stats['n_vars']

In [8]:
size_factors = torch.Tensor(eb_adata.obs['n_counts'])
torch.Tensor(eb_adata.obs['_scvi_individual'])

tensor([12., 12., 10.,  ..., 14.,  4.,  3.])

In [9]:
nb_lambda = torch.nn.Parameter(torch.zeros(n_inds, n_genes)) # make sure this is within a reasonable range
nb_theta = torch.nn.Parameter(torch.rand(n_inds, n_genes)) # make sure this is within a reasonable range

nb_mu = torch.mul(size_factors, torch.matmul(onehot, nb_lambda))

nb_model = NegativeBinomial(mu=nb_mu, theta=nb_theta)

SyntaxError: invalid syntax (3048322835.py, line 2)

In [59]:
def loss_fn(tensor_counts, nbmod, sizefactors, onehot):
    """
    tensor_counts: MxG tensor of counts, M=minibatch size, G=n genes
    nbmod: a negative binomial model
    """
    return -torch.sum(nbmod.log_prob(tensor_counts))

In [52]:
nb_model.mean

Parameter containing:
tensor([ 1.1404, -0.0899,  0.7298, -1.8453, -0.0250, -0.4794, -0.1434, -0.3173,
         0.9671, -0.9911,  0.3016, -0.1073,  0.9985, -0.0195,  0.9910, -0.7777,
         0.3140,  0.2133, -0.1201,  0.3605, -0.3140], requires_grad=True)

In [45]:
optim = torch.optim.Adam([nb_model.mu, nb_model.theta], lr=0.01)

In [ ]:
def train(x, labels):
    # run the model forward on the data
    label_pred = linear_reg_model(x).squeeze(-1)
    # calculate the mse loss
    loss = loss_fn(label_pred, labels.squeeze())
    # initialize gradients to zero
    optim.zero_grad()
    # backpropagate
    loss.backward()
    # take a gradient step
    optim.step()
    return loss